# Chapter 6. 요약

In [1]:
import sys
print(sys.version)

3.10.11 (main, Apr 20 2023, 19:02:41) [GCC 11.2.0]


In [2]:
!nvidia-smi

Thu Jun 29 23:45:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   38C    P8    22W / 170W |   2196MiB / 12288MiB |     17%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# GPU 메모리 사용량 확인
if device.type =="cuda":
    allocated_memory = torch.cuda.memory_allocated(device = device)
    cached_memory = torch.cuda.memory_reserved(device = device)

print(f"Allocated Memory:{allocated_memory / 1024 ** 2:.2f} MB")
print(f"Cached Memory: {cached_memory / 1024**2:.2f} MB")

Allocated Memory:0.00 MB
Cached Memory: 0.00 MB


**(서론)**

언젠가 한 번쯤 문서를 요약해야 할 때가 있습니다. 요약할 문서는 연구 논문이나 재무 실적 보고서 아니면 이메일 스레드일지도 모릅니다. 생각해보면 이런 작업에는 긴 단락을 이해하고, 관련 내용을 추론하고, 원래 문서의 주제를 통합해 유창한 텍스트를 생성하는 등 다양한 능력이 필요합니다. 또 기사를 정확하게 요약하는 방법과 법률 계약서를 요약하는 방법은 매우 다르기 때문에 정교한 수준의 도메인 일반화가 필요합니다. 이런 이유로 트랜스포머를 포함한 자연어 모델에게 텍스트 요약은 어려운 작업니다. 이런 어려움에도 불구하고 텍스트 요약은 도메인 전문가의 작업 속도를 크게 높이고 기업에서 내부 지식을 집약하고, 계약을 요약하고 소셜 미디어를 위한 자동 콘텐츠를 생성하는 등의 작업에 사용됩니다. 


이와 관련된 도전 과제를 이해하기 위해 이 장은 사전 훈련된 트랜스포머를 사용해 문서를 요약하는 방법을 알아보겠습니다. 요약은 입력과 출력이 텍스트인 고전적인 시퀀스-투-시퀀스 작업니다. 1장에서 보았듯이 요약에는 인코더-디코더 트랜스포머가 잘 맞습니다. 


이 장에서는 인코더-디코더 모델을 만들어 여러 사람이 주고 받은 대화를 간결하게 요약하겠습니다. 

하지만 그 전에 요약에 사용하는 대표적인 데이터 셋인 CNN/DAilyMail 말뭉치를 살펴보겠습니다.

## 6.1 CNN/DailyMail 데이터셋

CNN/DailyMail 데이터셋은 300,000개 뉴스 기사와 요약의 쌍으로 구성됐습니다. 요약은 CNN과 DailyMail이 기사에 첨부한 글머리 목록의 내용인데, 요약이 본문에서 추출되지 않고 추상적이라는 중요한 특징이 있습니다. 즉, 단순한 발췌가 아니라 새로운 문장으로 구성됐다는 이야기입니다. 이 데이터 셋은 허깅 페이스 허브(https://huggingface.co/datasets/cnn_dailymail)에서 제공합니다. 여기서는 요약을 위해 익명화 처리를 하지 않은 3.0.0 버전을 사용하겠습니다. 4장에서 서브셋을 선택한 대와 비슷한 방식으로 version 매개변수를 사용해 버전을 선택합니다. 

In [8]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", version = "3.0.0")
print(f"특성: {dataset['train'].column_names}")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /home/bread/.cache/huggingface/datasets/cnn_dailymail/default/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

특성: ['article', 'highlights', 'id']


In [16]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# GPU 메모리 사용량 확인
if device.type =="cuda":
    allocated_memory = torch.cuda.memory_allocated(device = device)
    cached_memory = torch.cuda.memory_reserved(device = device)

print(f"Allocated Memory:{allocated_memory / 1024 ** 2:.2f} MB")
print(f"Cached Memory: {cached_memory / 1024**2:.2f} MB")

Allocated Memory:0.00 MB
Cached Memory: 0.00 MB


In [14]:
import psutil

# 현재 프로세스의 메모리 정보 얻기
process = psutil.Process()
print(f"result : {process.memory_info().rss / 10**6:.2f} MB")  # 현재 사용 중인 메모리 양 (Resident Set Size)

result : 512.67 MB


이 데이터 셋은 세가지 특성이 있습니다. 

 - 뉴스 기사를 담은 article
 
 - 요약에 해당하는 highlights
 
 - 기사의 고유 아이디 id

In [17]:
# 기사 중 하나의 내용을 발췌함

sample = dataset["train"][1]
print(f""" 기사 (500개 문자 발췌, 총 길이: {len(sample["article"])}):""")
print(sample["article"][:500])
print(f'\n 요약 (길이: {len(sample["highlights"])}):')
print(sample["highlights"])

 기사 (500개 문자 발췌, 총 길이: 4051):
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

 요약 (길이: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


기사가 요약에 비해 매우 긴 경우도 있습니다. 이 경우 17배나 차이가 납니다. 대부분 트랜스포머 모델의 문맥 크기가 몇 단락에 해당하는 분량인 1,000개 토큰 정도로 제한되므로, 긴 기사는 트랜스포머 모델에 문제를 일으깁니다. 이를 처리하는 표준적이면서 가장 단순한 방법은 모델의 문맥 크기에 맞춰 텍스트를 자른 것입니다. 텍스트 끝부분에 중요한 정보가 있다면 사라지겠지만, 이는 모델 구조의 제약으로 생기는 불가피한 선택입니다.

## 6.2 텍스트 요약 파이프라인